In [ ]:
# Melakukan impor libraries yang diperlukan untuk visualisasi data.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.gridspec as gridspec
import math
import cv2
import time
import datetime

In [ ]:
# Melakukan impor MTCNN untuk deteksi wajah.
! pip install mtcnn
from mtcnn.mtcnn import MTCNN

In [ ]:
# Mencatat waktu dimulainya keseluruhan program visualisasi data.
global_start_time = time.time()

In [ ]:
# Membaca sekaligus menampilkan isi dari data training.
df_train = pd.read_csv('../input/bdc-satriadata2021/train.csv')
df_train

In [ ]:
# Mencatat waktu dimulainya pembacaan training images.
start_time = time.time()

# Mencatat index dari setiap training images.
img_train = {}
index_train = list(df_train.iloc[:,0])
subindex_train = [1, 2, 3]
filename_master_train = '../input/bdc-satriadata2021/Training/'

for index in index_train:
    # Membaca dan menyimpan training images.
    img_train[index] = {}
    img_index = str(index) + '/' + str(index) + '_'
    for subindex in subindex_train:
        img_subindex = str(subindex) + '.jpg'
        filename_train = filename_master_train + img_index + img_subindex
        img_train[index][subindex] = mpimg.imread(filename_train)

# Mencatat waktu berakhirnya pembacaan training images.
end_time = time.time()

# Menampilkan waktu eksekusi dari pembacaan training images.
execution_time = datetime.timedelta(seconds = end_time - start_time)
print("execution time: %s" % (execution_time))

In [ ]:
# Mendeklarasikan pendeteksi wajah.
detector = MTCNN()

In [ ]:
# Membaca sekaligus menampilkan informasi wajah pada data training.
info_train = pd.read_csv('../input/bdc-sd2021-data-tambahan/train_facial_data.csv')
info_train = info_train[info_train.iloc[:, 2] > -1].reset_index().drop('index', axis = 1)
info_train

In [ ]:
# Menentukan ukuran gambar dan margin gambar saat akan dirotasi.
adj_ratio = 0.25
pict_size = (128, 128)

In [ ]:
# Mencatat waktu dimulainya visualisasi training images.
start_time = time.time()

# Menyiapkan ukuran visualisasi data.
nrows = info_train.shape[0]
ncols = 4

# Menyiapkan tempat untuk penulisan judul.
title_frame = plt.figure(figsize = (6 * ncols, 10e-03), constrained_layout = False)
title_frame.patch.set_facecolor('white')
title_spec = gridspec.GridSpec(1, 1, figure = title_frame)

# Menuliskan judul.
main_title = title_frame.add_subplot(title_spec[0, 0])
main_title.set(xticks = [], yticks = [])
main_title.set_title('TRAINING IMAGES', fontsize = 72, color = '#007BA7')

# Menyiapkan dataframe untuk menampung informasi gambar.
train_gray = pd.DataFrame()
train_sobel_x = pd.DataFrame()
train_sobel_y = pd.DataFrame()
train_canny = pd.DataFrame()

for row in range(nrows):
    # Menyiapkan tempat untuk visualisasi data.
    frame = plt.figure(figsize = (6 * ncols, 8), constrained_layout = False)
    frame.patch.set_facecolor('white')
    spec = gridspec.GridSpec(1, ncols, figure = frame)
    
    nomor = info_train.iloc[row, 0]
    urutan_gambar = info_train.iloc[row, 1]
    jenis_kelamin = ['perempuan', 'laki-laki'][df_train.iloc[nomor-1, 1]]
    usia = df_train.iloc[nomor-1, 2]
    
    # Menyiapkan gambar untuk dirotasi.
    my_image = img_train[nomor][urutan_gambar].copy()
    x_coor, y_coor, width, height = info_train.iloc[row, 2:6]
    adj_x_coor = max(x_coor-int(adj_ratio*width), 0)
    adj_y_coor = max(y_coor-int(adj_ratio*height), 0)
    adj_width = int(width*(1+2*adj_ratio))
    adj_height = int(height*(1+2*adj_ratio))
    crop = my_image[adj_y_coor:adj_y_coor+adj_height, adj_x_coor:adj_x_coor+adj_width]
    
    # Melakukan rotasi.
    delta_x_eyes = int(info_train.iloc[row, 8] - info_train.iloc[row, 6])
    delta_y_eyes = int(info_train.iloc[row, 9] - info_train.iloc[row, 7])
    x_center = crop.shape[1] // 2
    y_center = crop.shape[0] // 2
    angle = math.degrees(math.atan(delta_y_eyes/delta_x_eyes))
    M = cv2.getRotationMatrix2D((x_center, y_center), angle, 1)
    out = cv2.warpAffine(crop, M, (crop.shape[1], crop.shape[0]))
    
    # Menentukan lokasi wajah yang baru.
    face_location = detector.detect_faces(out)
    if len(face_location):
        new_x, new_y, new_w, new_h = face_location[0]['box']
        main_face = out[new_y:new_y+new_h, new_x:new_x+new_w]
    else:
        # Definisi ulang gambar, jika deteksi wajah gagal dilakukan.
        crop = my_image[y_coor:y_coor+height, x_coor:x_coor+width]
        # Melakukan rotasi.
        delta_x_eyes = int(info_train.iloc[row, 8] - info_train.iloc[row, 6])
        delta_y_eyes = int(info_train.iloc[row, 9] - info_train.iloc[row, 7])
        x_center = crop.shape[1] // 2
        y_center = crop.shape[0] // 2
        angle = math.degrees(math.atan(delta_y_eyes/delta_x_eyes))
        M = cv2.getRotationMatrix2D((x_center, y_center), angle, 1)
        out = cv2.warpAffine(crop, M, (crop.shape[1], crop.shape[0]))
        main_face = out
    
    #  Menstandardisasi ukuran wajah dan warnanya.
    resized = cv2.resize(main_face, pict_size, interpolation = cv2.INTER_AREA)
    gray = cv2.cvtColor(resized, cv2.COLOR_RGB2GRAY)
    
    # Memunculkan gambar utama.
    add_image_0 = frame.add_subplot(spec[0, 0])
    add_image_0.imshow(gray, cmap = 'gray')
    title = str(row+1) + '. Gambar ' + str(nomor) + '_' + str(urutan_gambar)
    add_image_0.set_title(title, fontsize = 14)
    label_0 = 'jenis kelamin: ' + jenis_kelamin + '\n'
    label_1 = 'usia: ' + str(usia) + ' tahun' + '\n'
    add_image_0.set_xlabel(label_0 + label_1, fontsize = 12)
    add_image_0.set(xticks = [], yticks = [])
    
    # Memunculkan gambar hasil filter sobel x.
    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize = 3)
    add_image_1 = frame.add_subplot(spec[0, 1])
    add_image_1.imshow(sobel_x, cmap = 'gray')
    add_image_1.set(xticks = [], yticks = [])
    add_image_1.set_title('Sobel_X Filter', fontsize = 14)
    
    # Memunculkan gambar hasil filter sobel y.
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize = 3)
    add_image_2 = frame.add_subplot(spec[0, 2])
    add_image_2.imshow(sobel_y, cmap = 'gray')
    add_image_2.set(xticks = [], yticks = [])
    add_image_2.set_title('Sobel_Y Filter', fontsize = 14)
    
    # Memunculkan gambar hasil filter canny.
    canny = cv2.Canny(gray, 255/3, 255)
    add_image_3 = frame.add_subplot(spec[0, 3])
    add_image_3.imshow(canny, cmap = 'gray')
    add_image_3.set(xticks = [], yticks = [])
    add_image_3.set_title('Canny Filter', fontsize = 14)
    
    # Menyimpan informasi gambar dan filter pada dataframe.
    gray_df = pd.DataFrame(gray.reshape(1, gray.shape[0]*gray.shape[1])/255)
    train_gray = pd.concat([train_gray, gray_df], ignore_index = True)
    
    sobel_x_df = pd.DataFrame(sobel_x.reshape(1, sobel_x.shape[0]*sobel_x.shape[1])/255)
    train_sobel_x = pd.concat([train_sobel_x, sobel_x_df], ignore_index = True)
    
    sobel_y_df = pd.DataFrame(sobel_y.reshape(1, sobel_y.shape[0]*sobel_y.shape[1])/255)
    train_sobel_y = pd.concat([train_sobel_y, sobel_y_df], ignore_index = True)
    
    canny_df = pd.DataFrame(canny.reshape(1, canny.shape[0]*canny.shape[1])/255)
    train_canny = pd.concat([train_canny, canny_df], ignore_index = True)
    
    # Menampilkan keseluruhan visualisasi data.
    plt.show()

# Menyimpan seluruh informasi gambar ke dalam format .csv.
train_gray.to_csv('train_gray.csv', index = False)
train_sobel_x.to_csv('train_sobel_x.csv', index = False)
train_sobel_y.to_csv('train_sobel_y.csv', index = False)
train_canny.to_csv('train_canny.csv', index = False)

# Mencatat waktu berakhirnya visualisasi training images.
end_time = time.time()

# Menampilkan waktu eksekusi dari visualisasi training images.
execution_time = datetime.timedelta(seconds = end_time - start_time)
print("execution time: %s" % (execution_time))

In [ ]:
# Mencatat waktu berakhirnya keseluruhan program visualisasi data.
global_end_time = time.time()

# Menampilkan waktu eksekusi dari keseluruhan program visualisasi data.
total_execution_time = datetime.timedelta(seconds = global_end_time - global_start_time)
print("total execution time: %s" % (total_execution_time))